In [ ]:
import numpy as np
import nd2reader
import readlif
import xarray
import matplotlib
import matplotlib.pyplot as pl

In [156]:
import numpy as np
import pims
import nd2
import pandas as pd
import xarray
import dask.array as da
from tifffile import TiffFile

In [2]:
f = nd2.imread('data/A1_s3001.nd2', xarray = True, dask = True)

In [3]:
def readND2(filename):
    """
    
    """
    sequence = nd2.imread(filename, xarray = True, dask = True)
    frameT = sequence.metadata['experiment'][0].parameters.periodDiff.avg #this is the time per frame in ms
    scale = sequence.X[1] #this is the microns per pixel
    sequence['T'] = np.arange(len(sequence['T']))*frameT
    sequence.attrs = {'xyScale': float(scale),
                      'tScale': frameT}
    return(sequence)

In [4]:
seq = readND2('data/A1_s3001.nd2')
seq

<xarray.DataArray '_dask_block-7060f3470deffd836e17a4692376d751' (T: 5000,
                                                                  Y: 512, X: 512)>
dask.array<_dask_block, shape=(5000, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * X        (X) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * T        (T) float64 0.0 10.01 20.02 30.02 ... 5.001e+04 5.002e+04 5.003e+04
Attributes:
    xyScale:  0.122802855328918
    tScale:   10.007511707419608

In [117]:
def readLIF(filename):
    """
    
    """
    lifSeq = pims.Bioformats('data/21-03-31_ddm.lif', read_mode = 'jpype')
    xscale = lifSeq.metadata.PixelsPhysicalSizeX(0)
    tscale = lifSeq.metadata.PlaneDeltaT(0,1)
    lifXCoords = np.arange(0., lifSeq.shape[1]*xscale, xscale)
    lifYCoords = np.arange(0., lifSeq.shape[2]*xscale, xscale)
    lifTCoords = np.arange(0., lifSeq.shape[0]*tscale, tscale)
    ds = da.from_array(lifSeq)
    sequence = xarray.DataArray(data = ds, dims = ["T", "Y", "X"], coords = (lifTCoords, lifYCoords, lifXCoords), attrs = dict(xyScale = xscale, tScale = tscale))
    return sequence

In [118]:
readLIF('data/21-03-31_ddm.lif')

<xarray.DataArray 'array-31618e400289a7cdc7bed96af1d6f1ac' (T: 1000, Y: 1024,
                                                            X: 1024)>
dask.array<array, shape=(1000, 1024, 1024), dtype=uint8, chunksize=(500, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * T        (T) float64 0.0 0.012 0.024 0.036 0.048 ... 11.95 11.97 11.98 11.99
  * Y        (Y) float64 0.0 0.2045 0.409 0.6135 ... 208.6 208.8 209.0 209.2
  * X        (X) float64 0.0 0.2045 0.409 0.6135 ... 208.6 208.8 209.0 209.2
Attributes:
    xyScale:  0.20450566959921798
    tScale:   0.01200103759765625

In [248]:
tifSeq = pims.open('data/21-03-31_ddm_0.tif')
print(tifSeq._read_metadata(tifSeq._tiff)['ImageDescription'])
start = tifSeq._read_metadata(tifSeq._tiff)['ImageDescription'].find("interval=")+len("interval=")
tscale = float(tifSeq._read_metadata(tifSeq._tiff)['ImageDescription'][start:start+20])
tscale #this is the t resolution but may not be universal

ImageJ=1.53c
images=1000
frames=1000
unit=micron
finterval=0.012200199998915195
loop=false
min=62.0
max=255.0


0.012200199998915195

In [228]:
from PIL import Image
from PIL import TiffTags

In [218]:
image = Image.open('data/21-03-31_ddm_0.tif')
1./image.info['resolution'][0] #this is the x resolution but requires from PIL import Image

0.20450571071971899